<a href="https://colab.research.google.com/github/ydmatsd/funbgcex-colab2/blob/main/funbgcex-colab/FunBGCeX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/ydmatsd/funbgcex-colab2/blob/main/funbgcex-colab/logo.png?raw=true" height="60" align="top">

##<b>FunBGCeX (Fungal Biosynthetic Gene Cluster eXtractor) – Google Colab Version</b>
[**FunBGCeX**](https://github.com/ydmatsd/funbgcex) is a genome mining tool for fungal natural product discovery. It detects biosynthetic proteins based on our manually curated database of fungal natural product biosynthetic gene clusters (BGCs), [FunBGCs](http://staffweb1.cityu.edu.hk/ymatsuda/funbgcs/funbgcs.html). FunBGCeX can detect all possible BGCs from given fungal genomes and selectively extract BGCs according to the user's various needs. Currently available methods include:
* Extraction of BGCs encoding a homologue of a protein of the user's interest
* Extraction of BGCs harboring a putative self-resistance gene
* Extraction of BGCs lacking well-defined core biosynthetic enzyme genes (e.g., PKSs, NRPSs, and terpene synthases) — still under development

The extracted BGCs are provided in the GenBank format and can also be visualized using a web browser (You can access a sample output [here](http://staffweb1.cityu.edu.hk/ymatsuda/funbgcex/sample2/results.html)).
### <b>Citation</b>
[Tang, J., Matsuda, Y. *Nat. Commun.* **15**, 4312 (2024)](https://www.nature.com/articles/s41467-024-48771-7)

-----------

In [ ]:
#@title ## <b>Provide Input Information for BGC Extraction</b>
#@markdown ### Fill in the required settings below, then <b>ONLY</b> run this cell.
#@markdown Depending on your chosen input and extraction modes, you may need to:
#@markdown - Upload GenBank file(s)
#@markdown - Specify a folder path (on Google Drive or Colab local)
#@markdown - Upload a FASTA file (for `target` mode)
#@markdown - Select a Pfam domain (for `pfam` mode)

import os
import shutil
import time
from google.colab import files, drive
from IPython.display import display, clear_output
import ipywidgets as widgets

# ------------------------
# Input settings
# ------------------------
#@markdown **Choose how to provide input data**
InputMode = "upload" #@param ["upload", "path"]
#@markdown  - `upload`: Upload GenBank file(s). <b>The GenBank files need to have CDS features with a translation qualifier.</b>
#@markdown  - `path`: Specify an existing folder path (Google Drive or Colab local). <b>Ensure that no non-GenBank files are placed in this folder.</b>

#@markdown **Only for `path` as InputMode**

gbk_dir = "/content/gbk_dir" #@param {type:"string"}
#@markdown - Provide the path of the folder containing GenBank files (e.g., `/content/my_data`).
UseDrive = False #@param {type:"boolean"}
#@markdown - Tick here if you wish to use Google Drive (the path to the directory will be `/content/drive/MyDrive/[your directory name]`).

#@markdown **Choose a mode for BGC extraction**
mode = "all" #@param ["all", "target", "pfam", "ripps", "sre", "human"]
#@markdown  - `all`: Extract all possible BGCs.
#@markdown  - `target`: Extract BGCs that encode a homologue of protein(s) of your interest (a FASTA file is required).
#@markdown  - `pfam`: Extract BGCs that encode a protein with a protein family/domain (Pfam) of your interest.
#@markdown  - `ripps`: Extract BGCs that encode a possible precursor peptide for RiPP biosynthesis.
#@markdown  - `sre`: Extract BGCs with a possible self-resistance gene.
#@markdown  - `human`: Extract BGCs that encode a homologue of human disease-related proteins.

# ------------------------
# Global variables
# ------------------------
fasta_path = None
pfam = None

# ------------------------
# Functions
# ------------------------
def proceed_to_genbank_upload():
    """Handle GenBank upload / folder selection"""
    global gbk_dir

    clear_output()
    if InputMode == "upload":
        timestamp = int(time.time())
        gbk_dir = f"/content/gbk_upload_{timestamp}"
        os.makedirs(gbk_dir, exist_ok=True)
        print("📂 Please upload your GenBank file(s)")
        uploaded = files.upload()
        if not uploaded:
            print("⚠️ No GenBank file uploaded.")
            return
        for fname in uploaded.keys():
            dest_path = os.path.join(gbk_dir, fname)
            shutil.move(fname, dest_path)
            print(f"✅ GenBank file saved to {dest_path}")

    elif InputMode == "path":
        if UseDrive:
            drive.mount('/content/drive')
        print(f"📁 Using existing folder: {gbk_dir}")

def upload_fasta():
    """Upload a single FASTA file"""
    global fasta_path
    fasta_dir = "/content/fasta_input"
    os.makedirs(fasta_dir, exist_ok=True)
    print("📂 Please upload your FASTA file (single file).")
    uploaded = files.upload()
    if not uploaded:
        print("⚠️ No FASTA file uploaded.")
        return
    for fname in uploaded.keys():
        dest_path = os.path.join(fasta_dir, fname)
        shutil.move(fname, dest_path)
        fasta_path = dest_path
        print(f"✅ FASTA file saved to: {fasta_path}")

def select_pfam():
    """Select a Pfam domain"""
    global pfam

    pfam_list = ['2-Hacid_dh', '2-Hacid_dh_C', '2OG-FeII_Oxy', '2OG-FeII_Oxy_2', '2OG-FeII_Oxy_3', '2OG-Fe_Oxy_2', '3Beta_HSD', '3HCDH', '3HCDH_N', '4HBT', '4HBT_2', '4HBT_3', 'AA-adenyl-dom', 'AAA_16', 'AAA_19', 'AAA_21', 'AAA_22', 'AAA_23', 'AAA_25', 'AAA_29', 'AAA_30', 'AAA_33', 'AAT', 'AATase', 'AA_kinase', 'AA_permease', 'AA_permease_2', 'ABA3', 'ABC1', 'ABC2_membrane', 'ABC2_membrane_3', 'ABC2_membrane_7', 'ABC_ATPase', 'ABC_membrane', 'ABC_tran', 'ABC_trans_N', 'ABM', 'ACAS_N', 'ACC_central', 'ACPS', 'ACP_PCP', 'ACP_syn_III_C', 'ACT', 'ACT_7', 'ADC', 'ADH_N', 'ADH_N_2', 'ADH_zinc_N', 'ADH_zinc_N_2', 'ADP_ribosyl_GH', 'ADSL_C', 'AIG2_2', 'APH', 'ATEG_03636', 'ATP-grasp', 'ATP-grasp_4', 'ATP-synt_ab', 'ATP-synt_ab_N', 'ATP11', 'ATPgrasp_N', 'ATPgrasp_Ter', 'AXE1', 'Aa_trans', 'Abhydrolase_1', 'Abhydrolase_2', 'Abhydrolase_3', 'Abhydrolase_6', 'Acetyltransf_1', 'Acetyltransf_3', 'Acetyltransf_7', 'Acetyltransf_8', 'Aconitase', 'Aconitase_C', 'Acyl-CoA_dh_1', 'Acyl-CoA_dh_2', 'Acyl-CoA_dh_M', 'Acyl-CoA_dh_N', 'Acyl_transf_1', 'Acyl_transf_3', 'AflR', 'Aldedh', 'Aldo_ket_red', 'Aldolase_II', 'Aldose_epim', 'Alpha-amylase', 'Amidase', 'Amidohydro_1', 'Amidohydro_2', 'Amidohydro_3', 'Amino_oxidase', 'Aminotran_1_2', 'Aminotran_3', 'Aminotran_4', 'Aminotran_5', 'Ank', 'AnkA', 'AnkG', 'Ank_2', 'Ank_3', 'Ank_4', 'Ank_5', 'Anth_synt_I_N', 'Anthrone_oxy', 'ApbA', 'ApbA_C', 'Arginosuc_synth', 'AsR5', 'AsR6', 'Asn_synthase', 'Asp', 'Asp_Glu_race', 'Asp_aminotransf', 'AstC', 'AtaL', 'AtnE', 'BAAT_C', 'BBE', 'BD-FAE', 'Bac_luciferase', 'Beta-lactamase', 'Beta_elim_lyase', 'BfL6', 'BfL8', 'Biotin_carb_C', 'Biotin_carb_N', 'Biotin_lipoyl', 'CBM-like', 'CBS', 'CMAS', 'CMD', 'COesterase', 'CPSase_L_D2', 'CPSase_L_D3', 'CalD', 'Caps_synth', 'Carboxyl_trans', 'Carn_acyltransf', 'Catalase-rel', 'Catalase', 'CghF', 'Chal_sti_synt_C', 'Chal_sti_synt_N', 'Chlorophyllase2', 'Chorismate_bind', 'Citrate_synt', 'Clz3', 'Clz7', 'CoA_transf_3', 'Cofilin_ADF', 'Condensation', 'CosA', 'Cp1D', 'CpaE', 'CtvD', 'Cu-oxidase', 'Cu-oxidase_2', 'Cu-oxidase_3', 'Cu_amine_oxid', 'Cupin_2', 'Cys_Met_Meta_PP', 'Cyt-b5', 'Cytochrom_B561', 'DAHP_synth_1', 'DAO', 'DDE_1', 'DDE_3', 'DHDPS', 'DIOX_N', 'DIT1_PvcA', 'DJ-1_PfpI', 'DLH', 'DSBA', 'DUF1295', 'DUF1479', 'DUF1729', 'DUF1749', 'DUF1752', 'DUF1772', 'DUF1776', 'DUF1857', 'DUF1929', 'DUF1965', 'DUF1993', 'DUF2306', 'DUF2334', 'DUF2470', 'DUF2784', 'DUF3176', 'DUF3237', 'DUF3425', 'DUF3471', 'DUF3533', 'DUF4149', 'DUF4185', 'DUF4188', 'DUF4267', 'DUF4385', 'DUF4413', 'DUF4451', 'DUF5519', 'DUF6534', 'DUF6570', 'DUF659', 'DUF87', 'Dabb', 'Dala_Dala_lig_C', 'Dimer_Tnp_hAT', 'Dimerisation', 'Dimerisation2', 'Dynamin_N', 'Dyp_perox', 'ECH_1', 'ECH_2', 'EHN', 'ERG4_ERG24', 'EamA', 'EcKL', 'EcdJ', 'EfHP', 'Egh16-like', 'ElcH', 'Epimerase', 'EqxR', 'Esterase', 'EthD', 'F420_oxidored', 'F5_F8_type_C', 'FAA_hydrolase', 'FAD_binding_1', 'FAD_binding_2', 'FAD_binding_3', 'FAD_binding_4', 'FAD_binding_6', 'FAD_oxidored', 'FAE1_CUT1_RppA', 'FAS_I_H', 'FAS_N', 'FAS_meander', 'FA_hydroxylase', 'FMN_dh', 'FMN_red', 'FMO-like', 'FPSE_09189', 'FR47', 'FSH1', 'Fas_alpha_ACP', 'Fasciclin', 'Fe-ADH', 'Fe-ADH_2', 'Ferric_reduct', 'FhuF', 'Flavodoxin_1', 'Flavodoxin_2', 'Fn3-like', 'FncF', 'FolB', 'FrsA-like', 'FrzJ', 'Fsa2', 'FscB', 'FtsJ', 'FtsK_SpoIIIE', 'Fungal_trans', 'Fungal_trans_2', 'GATase', 'GATase_6', 'GATase_7', 'GBBH-like_N', 'GDP_Man_Dehyd', 'GFO_IDH_MocA', 'GFO_IDH_MocA_C', 'GGPS', 'GH3', 'GH43_C2', 'GIDA', 'GMC_oxred_C', 'GMC_oxred_N', 'GO-like_E_set', 'GST_C', 'GST_C_2', 'GST_C_3', 'GST_C_5', 'GST_N', 'GST_N_2', 'GST_N_3', 'GST_N_4', 'GTP_cyclohydroI', 'G_glu_transpept', 'GliK', 'GliN', 'Gln-synt_C', 'Glu_dehyd_C', 'Gly_transf_sug', 'Glyco_hydro_11', 'Glyco_hydro_3', 'Glyco_hydro_31', 'Glyco_hydro_3_C', 'Glyco_hydro_43', 'Glyco_hydro_cc', 'Glyco_tran_28_C', 'Glyco_transf_28', 'Glyoxal_oxid_N', 'Glyoxalase', 'Glyoxalase_3', 'Glyoxalase_4', 'Glyoxalase_6', 'Gp_dh_C', 'Gp_dh_N', 'Gpr1_Fun34_YaaH', 'GsfR1', 'HAD_2', 'HD', 'HI0933_like', 'HLH', 'HMG-CoA_red', 'HMGL-like', 'HMG_CoA_synt_C', 'HMG_CoA_synt_N', 'HPIH', 'HSP20', 'HTH_51', 'HTH_Tnp_Tc5', 'HTH_psq', 'Haem_degrading', 'HbpS-like', 'Helitron_like_N', 'Helo_like_N', 'Hemerythrin', 'Hemocyanin_C', 'Hemocyanin_M', 'Hexapep', 'HimD', 'Histone', 'Histone_H2A_C', 'Homeobox_KN', 'Homeodomain', 'Homoserine_dh', 'HpcH_HpaI', 'Hydrolase', 'Hydrolase_4', 'Hydrolase_like', 'ICMT', 'IDO', 'ILVD_EDD', 'IMPDH', 'IPPT', 'IPT', 'Iso_dh', 'Isochorismatase', 'IucA_IucC', 'K_oxygenase', 'Kdo', 'Kelch_1', 'Kelch_2', 'Kelch_3', 'Kelch_4', 'Kelch_6', 'LCM', 'LIP', 'Lactamase_B', 'Lactamase_B_2', 'Lactamase_B_3', 'LcsT', 'LigB', 'Lipocalin_5', 'Lipocalin_9', 'LolU', 'LtmS', 'Lumazine_bd_2', 'Lyase_1', 'Lyase_aromatic', 'Lycopene_cycl', 'Lys_Orn_oxgnase', 'Lysine_decarbox', 'M20_dimer', 'MARVEL', 'MBOAT_2', 'MFS_1', 'MFS_1_like', 'MFS_2', 'MFS_4', 'MFS_5', 'MMR_HSR1', 'MPAB_Lcp_cat', 'MTS', 'MULE', 'MafB19-deam', 'MaoC_dehydrat_N', 'MaoC_dehydratas', 'MdpA', 'Melibiase_2', 'Melibiase_C', 'MetW', 'Methyltransf_10', 'Methyltransf_11', 'Methyltransf_12', 'Methyltransf_16', 'Methyltransf_2', 'Methyltransf_21', 'Methyltransf_23', 'Methyltransf_24', 'Methyltransf_25', 'Methyltransf_28', 'Methyltransf_3', 'Methyltransf_31', 'Methyltransf_32', 'Methyltransf_33', 'Methyltransf_4', 'Methyltransf_7', 'Methyltransf_9', 'Mito_carr', 'MmgE_PrpD', 'MmgE_PrpD_C', 'MscL', 'Myb_DNA-bind_6', 'Myb_DNA-bind_7', 'Myb_DNA-binding', 'NACHT', 'NAD_binding_1', 'NAD_binding_10', 'NAD_binding_11', 'NAD_binding_2', 'NAD_binding_3', 'NAD_binding_4', 'NAD_binding_6', 'NAD_binding_8', 'NAD_binding_9', 'NMO', 'NRPS_C', 'NRPS_CT', 'NRPS_E', 'NRPS_term_dom', 'Nitroreductase', 'NmrA', 'NnrU', 'NodS', 'NtCtMGAM_N', 'NvfI', 'O-FucT', 'OATP', 'OPT', 'OphC', 'Orn_Arg_deC_N', 'Orn_DAP_Arg_deC', 'OtaR1', 'Oxidored_FMN', 'Oxysterol_BP', 'P5CR_dimer', 'PAP2_3', 'PBP', 'PCMT', 'PDR_CDR', 'PEMT', 'PEP_mutase', 'PHM7_cyt', 'PIF1', 'PIGO_PIGG', 'PKS_AT', 'PKS_KS', 'PKS_NRPS_MT', 'PK_Tyr_Ser-Thr', 'PPR_2', 'PSDC', 'PS_Dcarbxylase', 'PTR2', 'P_proprotein', 'PatF', 'PatJ', 'Patched', 'PaxC', 'PbcA', 'Pec_lyase', 'PenF', 'Peptidase_M19', 'Peptidase_M20', 'Peptidase_M24', 'Peptidase_M3', 'Peptidase_S10', 'Peptidase_S15', 'Peptidase_S28', 'Peptidase_S41', 'Peptidase_S8', 'Peptidase_S9', 'Peptidase_S9_N', 'Peroxidase_2', 'PfB', 'Phe_hydrox_dim', 'PhomB', 'Phosphodiest', 'PhyH', 'Pkinase', 'Polyketide_cyc2', 'Polysacc_deac_1', 'Polysacc_synt_2', 'Prefoldin', 'Prenyltrans', 'Pribosyltran', 'Pro_CA', 'Pro_isomerase', 'Proteasome', 'PrpF', 'PydY', 'Pyr4', 'Pyr_redox', 'Pyr_redox_2', 'Pyr_redox_3', 'Pyridoxal_deC', 'Questin_oxidase', 'RSN1_7TM', 'RSN1_TM', 'RTA1', 'RmlD_sub_bind', 'RsgA_GTPase', 'Rua1_C', 'SAT2', 'SET', 'SGL', 'SMC_N', 'SQHop_cyclase_C', 'SQHop_cyclase_N', 'SQS_PSY', 'Sacchrp_dh_NADP', 'SalTPS', 'Say1_Mug180', 'Scytalone_dh', 'Sdh_cyt', 'SdnG', 'SdnJ', 'Semialdhyde_dh', 'Shikimate_DH', 'SmbD', 'SnoaL', 'SnoaL_2', 'SnoaL_4', 'Spd14', 'SrdD', 'Steroid_dh', 'Sterol-sensing', 'Sterol_MT_C', 'Str_synth', 'Sugar_tr', 'Sulfotransfer_4', 'T2SSE', 'TFIIE_alpha', 'TIM', 'TPMT', 'TPP_enzyme_C', 'TPP_enzyme_M', 'TPP_enzyme_N', 'TPR_2', 'TP_methylase', 'TRAM1', 'TRAM_LAG1_CLN8', 'TRI12', 'TRI5', 'TRI9', 'TTL', 'Tannase', 'TauD', 'Tcf25', 'TdiC', 'Terpene_syn_C_2', 'Terpene_synth_C', 'Thi4', 'Thioesterase', 'Thiolase_N', 'Tmemb_14', 'TqaK', 'Transferase', 'Transketolase_C', 'Transketolase_N', 'Transp_cyt_pur', 'Tri14', 'Tri3', 'Tri6', 'Trp_DMAT', 'Trp_halogenase', 'Trt14', 'Trt5', 'Trypsin', 'Trypsin_2', 'TwmC', 'Tyrosinase', 'UDPGT', 'UNC-93', 'UbiA_PT', 'UbiA_TC', 'Ubie_methyltran', 'UstYa', 'VicA', 'VicYx', 'VidS', 'VniA', 'VrtF', 'WD40', 'XenC', 'XenD', 'YCII', 'Zn_clus', 'adh_short', 'adh_short_C2', 'bZIP_1', 'dCMP_cyt_deam_1', 'fPKS_DH', 'fPKS_ER', 'fPKS_KR', 'fPKS_PT', 'fPKS_R', 'fPKS_TH', 'fn3_3', 'p450', 'peroxidase', 'tRNA-synt_1c', 'tRNA-synt_1c_C', 'tRNA-synt_2', 'tRNA-synt_2c', 'tRNA-synt_2d', 'tRNA_anti-codon', 'zf-C2H2', 'zf-C6H2', 'zf-H2C2_2', 'zf_CCCH_4']

    pfam_widget = widgets.Combobox(
        placeholder='Type to search Pfam...',
        options=pfam_list,
        description='Pfam:',
        ensure_option=True,
        disabled=False
    )
    confirm_button = widgets.Button(description="Confirm Pfam")

    def on_confirm_clicked(b):
        global pfam
        pfam = pfam_widget.value
        if pfam.strip() == "":
            print("⚠️ Please select a Pfam domain before confirming.")
            return
        clear_output()
        print(f"✅ Pfam selected: {pfam}")
        proceed_to_genbank_upload()

    confirm_button.on_click(on_confirm_clicked)
    display(pfam_widget, confirm_button)

# ------------------------
# Main logic
# ------------------------
if mode == "target":
    upload_fasta()
    proceed_to_genbank_upload()
elif mode == "pfam":
    select_pfam()
else:
    proceed_to_genbank_upload()

#Output directory
results_dir = "/content/result"

##▶️ **NEXT STEP:** After completing the first step, move to the next cell, then go to `Runtime` -> `Run after`. Adjust the optional settings below if needed.


In [ ]:
#@title Optional settings
gap_allowed = 2500 #@param {type:"integer"}
#@markdown - Maximum allowed gap (in bp) between two adjacent genes in a cluster, if no correlation is detected between the two genes (Default: 2500)
max_bgc_gap = 15000 #@param {type:"integer"}
#@markdown - Maximum allowed gap (in bp) between two adjacent genes, if they only have a weak correlation (Default: 15000)
min_prot_len = 200 #@param {type:"integer"}
#@markdown - Minimum protein length considered for biosynthetic protein detection (Default: 200)
num_of_genes_checked = 20 #@param {type:"integer"}
#@markdown - Number of genes around the core/target genes to be checked (Default: 20)
min_identity = 50 #@param {type:"integer"}
#@markdown - Minimum identity (in %) required to be considered as a duplicated protein (Default: 50)
additional_genes = 0 #@param {type:"integer"}
#@markdown - Number of additional genes to be included outside a predicted BGC (Default: 0)
noCore = False #@param {type:"boolean"}
#@markdown - Tick here if you wish to extract BGCs without a core protein as well. This option must be used with either `target` or `pfam` mode.
noUstY = False #@param {type:"boolean"}
#@markdown - Tick here if you wish to extract RiPP BGCs without a UstY homologue(s).
noKexB = False #@param {type:"boolean"}
#@markdown - Tick here if you wish to extract RiPP BGCs without a precursor peptide with KexB cleavage sites.
NeedCSV = False #@param {type:"boolean"}
#@markdown - Tick here if you wish to to include analysis CSV files in the output.

In [ ]:
#@title Install dependencies
import os
import shutil
!sudo apt-get install hmmer diamond-aligner
!pip install biopython
!pip install natsort
!pip install tensorflow
if os.path.isdir("/content/funbgcex") == False:
  !git clone https://github.com/ydmatsd/funbgcex-colab2.git
  shutil.move("/content/funbgcex-colab2/funbgcex-colab/funbgcex","/content")

In [ ]:
#@title Extract BGCs
import os
import sys
from funbgcex.BGCeXtractor import BGCeXtractorMain

funbgcex_path = "/content/funbgcex"

#Accept input files
if os.path.isdir(gbk_dir) == False:
    sys.exit("The input directory does not exist.")
if (mode == "all" or mode == "ripps" or mode == "sre" or mode == "human") and noCore:
    sys.exit("'no_core' can only be used with the target or pfam mode.")
if additional_genes < 0:
    sys.exit("Please provide a non-negative int value for additional_genes.")
if mode == "all" or mode == "ripps" or mode == "sre" or mode == "human":
    query = "none"
elif mode == "target":
    query = fasta_path
    if query == "none":
        sys.exit("Please provide a fasta file.")
    if os.path.isfile(query) == False:
        sys.exit("The fasta file does not exist.")
elif mode == "pfam":
    PfamList = []
    allHMM = f"{funbgcex_path}/data/hmm/fungalSM/fungalSM.hmm"
    with open(allHMM,"r") as hmm:
        for line in hmm:
            if line.startswith("NAME"):
                name = line.split(" ")[-1].replace("\n","")
                PfamList.append(name)
    query = pfam
    if query not in PfamList:
        if query != "none":
            print(f"{query} does not exist in the database.")
            ToBeContinued = True
            while ToBeContinued:
                answer = input("Do you want to check the available protein family names? y/n: ")
                if answer == "Y" or answer == "y":
                    print(PfamList)
                    ToBeContinued = False
                    sys.exit()
                elif answer == "N" or answer == "n":
                    ToBeContinued = False
                else:
                    pass
            sys.exit()
        else:
            sys.exit("Please provide the name of a protein family to be used for BGC extraction.")
else:
    sys.exit("Please choose 'all', 'ripps', 'target', 'pfam', 'sre', or 'human' as an extraction mode.")

"""
Start BGC extraction
"""
workers = 1
BGCeXtractorMain(gbk_dir,results_dir,mode,query,gap_allowed,max_bgc_gap,min_prot_len,num_of_genes_checked,min_identity,additional_genes,noCore,noUstY,noKexB,workers,NeedCSV)


In [ ]:
#@title Download results
import shutil
from google.colab import files

shutil.make_archive("/content/results", 'zip', results_dir)
shutil.rmtree(results_dir)
shutil.rmtree(gbk_dir)
files.download("/content/results.zip")